# In Today's task you will

- Implement Linear (also called Dense, Fully-Connected) layer as a Perceptron.
- Allow your solution to stack multiple layers to form MLP network.
- Perform forward propagation through your network.

This (and later) template implementation is similar to Pytorch framework.


## Task 1a:

Declare a simple perceptron (Linear layer) that inherits defined class Module - it is here, to help you store all network layers.

The simple perceptron should be constructed of:

1. Input features
2. Followed by 1 Linear Layer with "single neuron"
3. Activation function

4. Perform forward pass for the example feature vectors `xInput1` and `xInput2` of `size = 10` features.
   Use prepared plot to view the results. (Repeat the process using all 4 activation functions.)


In [68]:
# Import
import numpy as np
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from collections import OrderedDict

### Module

All deep learning frameworks have usually one elementary building block.
In our project, we follow the structure of the pytorch, so the elementary building block is called **`Module`**.
Now, it is pretty simple, but it will get more complex and more useful...
You can see function `.backward` that will later contain the partial derivations of chain rule for backward pass and parameter optimization.


In [69]:
class Module:
    def __init__(self):
        self.modules = OrderedDict()

    def add_module(self, module, name:str):
        if hasattr(self, name) and name not in self.modules:
            raise KeyError("attribute '{}' already exists".format(name))
        elif '.' in name:
            raise KeyError("module name can't contain \".\"")
        elif name == '':
            raise KeyError("module name can't be empty string \"\"")
        self.modules[name] = module

    def forward(self, *args, **kwargs) -> np.ndarray:
        pass

    def backward(self, *args, **kwargs):
        pass

    def __call__(self, *args, **kwargs):
        return self.forward(*args, **kwargs)


## Linear Layer

In the lecture, we talked about a Perceptron and Single Layer Perceptron as an object with weight for every input value.
In the frameworks, the "Fully connected layer" is implemented in Matrix Algebra.

Also, the activation function and layer logic are separated for easier backward propagation (chain rule) and optimization (The topic of 2nd+3rd lecture).

(If you want to know more, you can go to the lecture, or you can take a look on the implementation of forward and backward propagation on your own.)


In [70]:
#------------------------------------------------------------------------------
#   Linear class
#------------------------------------------------------------------------------
class Linear(Module):
    def __init__(self, in_features, out_features):
        super(Linear, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.W = np.random.randn(out_features, in_features)
        self.b = np.zeros((out_features,))

    def forward(self, input: np.ndarray) -> np.ndarray:
        # >>>>>>>>> add here
        return np.matmul(self.W,input) + self.b
        # <<<<<<<<<

    def backward(self, dNet):
        pass


## Activations

The definitions for Sigmoid, Tanh, ReLU, and LeakyReLU activation functions with forward and backward pass.
Implement the forward pass. (for now, you leave the backward pass on `pass`)


In [71]:
#------------------------------------------------------------------------------
#   SigmoidActivationFunction class
#------------------------------------------------------------------------------
class Sigmoid(Module):
    def __init__(self):
        super(Sigmoid, self).__init__()

    def forward(self, input: np.ndarray) -> np.ndarray:
        # >>>>>>>>> add here
        return 1 / (1 + np.exp(-input))
        # <<<<<<<<<

    def backward(self, dNet):
        pass

#------------------------------------------------------------------------------
#   HyperbolicTangentActivationFunction class
#------------------------------------------------------------------------------
class Tanh(Module):
    def __init__(self):
        super(Tanh, self).__init__()

    def forward(self, input: np.ndarray) -> np.ndarray:
        # >>>>>>>>> add here
        # e^x − e^−x / e^x + e^−x
        return (np.exp(input) - np.exp(-input)) / (np.exp(input) + np.exp(-input))
        # <<<<<<<<<

    def backward(self, dNet):
        pass

#------------------------------------------------------------------------------
#   RELUActivationFunction class
#------------------------------------------------------------------------------
class ReLU(Module):
    def __init__(self):
        super(ReLU, self).__init__()

    def forward(self, input: np.ndarray) -> np.ndarray:
        # >>>>>>>>> add here
        # return np.maximum(input,0)
        return np.where(input>0,input,0)

        # <<<<<<<<<

    def backward(self, dNet):
        pass

#------------------------------------------------------------------------------
#   LeakyRELUActivationFunction class
#------------------------------------------------------------------------------
class LeakyReLU(Module):
    # >>>>>>>>> add something here
    def __init__(self):
        super(LeakyReLU, self).__init__()

    def forward(self, input: np.ndarray) -> np.ndarray:
        return np.where(input>0,input,0.01*input)
    # <<<<<<<<<<<
    def backward(self, dNet):
        pass

### Plotting the functions

Verify your implementations of Activation functions - do your graphs look like they should?


In [72]:
activationsInput = np.linspace(-4,4,100)

sigmoid = Sigmoid()
y = sigmoid.forward(activationsInput)

fig = make_subplots(rows=2, cols=2)

fig.add_trace(
    go.Scatter(x=activationsInput, y=y, name='Sigmoid'),
    row=1, col=1
)

tanh = Tanh()
y = tanh.forward(activationsInput)
fig.add_trace(
    go.Scatter(x=activationsInput, y=y, name='Tanh'),
    row=1, col=2
)

relu = ReLU()
y = relu(activationsInput)
fig.add_trace(
    go.Scatter(x=activationsInput, y=y, name='ReLU'),
    row=2, col=1
)

leakyrelu = LeakyReLU()
y = leakyrelu(activationsInput)
fig.add_trace(
    go.Scatter(x=activationsInput, y=y, name='LeakyReLU'),
    row=2, col=2
)

fig.update_layout(height=600, width=800, title_text="Activation functions")
fig.show()


### Perceptron feed forward

Model your Perceptron.
Define and initialize perceptron with "1 neuron"!
Feed `xInput1` and `xInput2` to the perceptron and print the results.


In [90]:
# generate 10 inputs
xInput1 = np.arange(10)
xInput2 = np.random.random(10)

# >>>>>>>>> Initialize Your Perceptron Here
# definicia triedy perceptronu
class Perceptron:
    def __init__(self,in_features,out_features):
        self.module = Module()
        self.module.add_module(Linear(in_features,out_features),'linear')
        self.module.add_module(Sigmoid(),'sigmoid')
        self.module.add_module(LeakyReLU(),'leakyReLu')

    def run(self,input,activation_function=None):      
        # linearne vynasobenie vstupu a vah
        z = self.module.modules['linear'](input)
        if activation_function:
            # aplikovanie aktivacnej funkcie
            a = self.module.modules[activation_function](z)

        print(f'input ->\n{input}')
        print(f'out(linear)= {z}')
        if activation_function:
            print(f'activate({activation_function})= {a}\n')

# instance of perceptron
perceptron = Perceptron(xInput1.shape[0],1)
perceptron.run(xInput1)
perceptron.run(xInput2)
# <<<<<<<<< Use as many lines as you need

input ->
[0 1 2 3 4 5 6 7 8 9]
out(linear)= [7.08030911]
input ->
[0.43122839 0.78619362 0.8689177  0.30392903 0.11589691 0.76456343
 0.29589361 0.95018881 0.2162259  0.59393354]
out(linear)= [-0.30547003]


Your Perceptron with an Activation function
Use previously defined perceptron and use its output as input for the activation function sigmoid and LeakyReLU.
Feed `xInput1` and `xInput2` to the perceptron, print and observe the results.


In [86]:
# >>>>>>>>> Initialize activations and feed them after perceptron
perceptron.run(xInput1,'sigmoid')
perceptron.run(xInput2,'leakyReLu')
# <<<<<<<<< Use as many lines as you need

input ->
[0 1 2 3 4 5 6 7 8 9]
out= [16.20337136]
activate(sigmoid)= [0.99999991]

input ->
[0.67021939 0.34365719 0.74184874 0.30356569 0.64048098 0.59542928
 0.24941908 0.54998812 0.2437     0.39228596]
out= [2.47741552]
activate(leakyReLu)= [2.47741552]



## Task 1b:

Finish the implementation of class `Model` - finish the call of forward feed.
Declare a simple model consisting of:

1.  Input Layer
2.  3 Linear Layers with arbitrary number of neurons
3.  Output Linear Layer with 1 neuron.

...and activation functions to add non-linearity

Declare your own input vector with 16 features.
Perform forward pass through the network and print the results.

### Model class

Implementation of the **`Model`** class.
Define its forward function - the implementation of forward and backward pass is sensitive to the order of called operations.
Each Layer(module) of type **`Module`** can be saved to the attribute **`Module.modules`** using the **`add_module`** method.


In [50]:
#------------------------------------------------------------------------------
#   Model class
#------------------------------------------------------------------------------
class Model(Module):
    def __init__(self):
        super(Model, self).__init__()

    def forward(self, input):
        # >>>>>>>>> add here
        return
        # <<<<<<<< do something beautiful... and simple

    def backward(self, dA: np.ndarray):
        pass

In [51]:
model = Model()
# >>>>>>>>> Build the model architecture with 3 layers (input, hidden, output) that can process - feed forward the xInput1 and xInput2

In [ ]:
# print(model. ...)
# print(model. ...)